# Mediapipe

In [1]:
!pip install opencv-python mediapipe

In [14]:
import cv2
import mediapipe as mp

# 얼굴 검출 모델
mp_face_detector = mp.solutions.face_detection
mp_drawing = mp.solutions.drawing_utils

face_detector = mp_face_detector.FaceDetection(model_selection=0, min_detection_confidence=0.5)

In [20]:
cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, frame = cap.read()

    img_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    results = face_detector.process(img_rgb)

    # print(results.detections)

    if results.detections:
        for detection in results.detections:
            mp_drawing.draw_detection(frame, detection)

    cv2.imshow('Face Detection', frame)

    if cv2.waitKey(1) & 0xFF == 27:
        break

cap.release()
cv2.destroyAllWindows()

error: OpenCV(4.11.0) D:\a\opencv-python\opencv-python\opencv\modules\highgui\src\window.cpp:1301: error: (-2:Unspecified error) The function is not implemented. Rebuild the library with Windows, GTK+ 2.x or Cocoa support. If you are on Ubuntu or Debian, install libgtk2.0-dev and pkg-config, then re-run cmake or configure script in function 'cvShowImage'


# 2. 얼굴 인식 모자이크

In [ ]:
cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, frame = cap.read()

    img_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    img_h, img_w, _ = img_rgb.shape

    results = face_detector.process(img_rgb)

    if results.detections:
        for detection in results.detections:
            bbox = detection.location_data.relative_bounding_box

            x1 = int(max(bbox.xmin, 0) * img_w)
            y1 = int(max(bbox.ymin, 0) * img_h)
            width = int(bbox.width * img_w)
            height = int(bbox.height * img_h)
            x2 = min(x1 + width, img_w)
            y2 = min(y1 + height, img_h)

            face = img_rgb[y1:y2, x1:x2]
            face = cv2.resize(face, (10, 10), interpolation=cv2.INTER_LINEAR)
            face = cv2.resize(face, (x2-x1, y2-y1), interpolation=cv2.INTER_LINEAR)

            img_rgb[y1:y2, x1:x2] = face
            
    img_rgb = cv2.cvtColor(img_rgb, cv2.COLOR_RGB2BGR)
    cv2.imshow('Face Detection', img_rgb)

    if cv2.waitKey(1) & 0xFF == 27:
        break

cap.release()
cv2.destroyAllWindows()

error: OpenCV(4.11.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.cpp:199: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'


# 3. FaceMesh

In [ ]:
import cv2
import mediapipe as mp
import numpy as np

# 얼굴 인식 및 랜드마크 검출을 위한 초기화
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_face_mesh = mp.solutions.face_mash

face_mesh = mp_face_mesh.FaceMesh()

In [ ]:
cap = cv2.VideoCapture(0)

with mp_face_mesh.FaceMesh(
    max_num_faces=1,
    refine_landmarks=True,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5
) as face_mesh:
    
    while cap.isOpened():
        ret, frame = cap.read()

        if not ret:
            break

        frame = cv2.flip(frame, 1)
        image = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)

        image.flags.writeable = False
        results = face_mesh.process(image)
        image.flags.writeable = True

        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        if results.multi_face_landmarks:
            for face_landmark in results.multi_face_landmarks:

                mp_drawing.draw_landmarks(
                    image=image,
                    landmark_list=face_landmark,
                    connections=mp_face_mesh.FACEMESH_TESSELATION,
                    landmark_drawing_spec=None,
                    connection_drawing_spec=mp_drawing_styles.get_default_face_mesh_tesselation_style()
                )

                mp_drawing.draw_landmarks(
                    image=image,
                    landmark_list=face_landmark,
                    connections=mp_face_mesh.FACEMESH_IRISES,
                    landmark_drawing_spec=None,
                    connection_drawing_spec=mp_drawing_styles.get_default_face_mesh_iris_connections_style()
                )
    
        cv2.imshow('Face Mesh', image)

        if cv2.waitKey(1) & 0xFF == 27:
            break

cap.release()
cv2.destroyAllWindows()

# 4. 필터 씌우기

In [ ]:
import cv2
import mediapipe as mp
import numpy as np

mp_draw = mp.solutions.drawing_utils
mp_face_mesh = mp.solutions.face_mesh

face_mesh = mp_face_mesh.FaceMesh()

In [ ]:
mask_img = cv2.imread('mini.png', cv2.IMREAD_UNCHANGED)

In [ ]:
def face_overlay(background_img, overlay_img, x, y, overlay_size=None):
    try:
        bg_img = background_img.copy()

        if bg_img.shape[2] == 3:
            bg_img = cv2.cvtColor(bg_img, cv2.COLOR_BGR2BGRA)

        if overlay_size is not None:
            overlay_img = cv2.resize(overlay_img.copy(), overlay_size)

        b, g, r, a = cv2.split(overlay_img)
        mask = cv2.medianBlur(a, 5)

        h, w, _ = overlay_img.shape
        roi = bg_img[int(y - h/2):int(y +h/2), int(x - w/2):int(x +w/2)]
        img_bg1 = cv2.bitwise_and(roi, roi, mask=cv2.bitwise_not(mask))
        img_bg2 = cv2.bitwise_and(overlay_img.copy(), overlay_img.copy(), mask=mask)
        bg_img[int(y -h/2):int(y + h/2), int(x - w/2):int(x + w/2)] = cv2.add(img_bg1, img_bg2)

        bg_img = cv2.cvtColor(bg_img, cv2.COLOR_BGRA2BGR)
        return bg_img
    
    except:
        return background_img


In [ ]:
cap = cv2.VideoCapture(0)

while True:
    ret, img = cap.read()

    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    result = face_mesh.process(img_rgb)
    img_h, img_w, img_c = img.shape

    if result.multi_face_landmarks:
        for face_landmark in result.multi_face_landmarks:
            xy_point = []

            for c, lm in enumerate(face_landmark.landmark):
                xy_point.append([lm.x, lm.y])
                img = cv2.circle(img, (int(lm.x * img_w), int(lm.y * img_h)), 1, (255, 0, 0))                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        

            top_left = np.min(xy_point, axis=0)
            bottom_right = np.max(xy_point, axis=0)                                                                                                                                                                                 

    if cv2.waitKey(1) == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()